In [ ]:
import sys
!{sys.executable} -m pip install -q anthropic google-genai

In [ ]:
from anthropic import Anthropic       # pip install anthropic
from google import genai               # pip install google-genai
import time

In [ ]:
import getpass

# Collect API keys via interactive input (no .env file needed)
ANTHROPIC_API_KEY = getpass.getpass("Enter your Anthropic API key: ")
GOOGLE_API_KEY = getpass.getpass("Enter your Google/Gemini API key: ")

# Initialize clients
anthropic_client = None
gemini_client = None

# 1. Setup Claude
if ANTHROPIC_API_KEY:
    try:
        anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)
        print("✅ Claude ready")
    except Exception as e:
        print(f"❌ Claude failed: {e}")

# 2. Setup Gemini (new google-genai SDK)
if GOOGLE_API_KEY:
    try:
        gemini_client = genai.Client(api_key=GOOGLE_API_KEY)
        print("✅ Gemini ready")
    except Exception as e:
        print(f"❌ Gemini failed: {e}")

In [ ]:
def call_claude(prompt, max_tokens=1024):
    if not anthropic_client:
        return "[Claude unavailable]"
    try:
        msg = anthropic_client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=max_tokens,
            messages=[{"role": "user", "content": prompt}]
        )
        return msg.content[0].text
    except Exception as e:
        return f"[Error: {e}]"

def call_gemini(prompt):
    if not gemini_client:
        return "[Gemini unavailable]"
    try:
        response = gemini_client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
        )
        return response.text
    except Exception as e:
        return f"[Error: {e}]"

def compare(prompt, label=""):
    print(f"\n{'='*80}\n{label}\n{'='*80}\n")
    print(f"PROMPT:\n{'-'*80}\n{prompt}\n{'-'*80}\n")
    
    print("🔵 CLAUDE:")
    print(call_claude(prompt))
    time.sleep(1)
    
    print("\n🟢 GEMINI:")
    print(call_gemini(prompt))

***Defining Between Vague Prompt Vs Specific Instructions***

In [ ]:
report = """Q3 2024: Revenue $45.2M (+23% YoY). SaaS: $32M (71%). 
Gross margin: 68%. OpEx: $28M (+18%). CAC: $1,200 (from $1,450). 
142 new enterprise deals. MRR: $10.2M (+19%). Churn: 2.1%. 
Challenges: EU regulatory delays, AI features delayed to Q1 2025."""

print("\n" + "="*80)
print("EXPERIMENT 1: VAGUE VS SPECIFIC INSTRUCTIONS")
print("="*80)

# Bad prompt
compare(
    f"Summarize this report:\n\n{report}",
    "❌ BAD: Vague request"
)

# Good prompt
compare(
    f"""### ROLE ###
You are a financial analyst for C-suite executives.

### TASK ###
Summarize this quarterly report with ONLY critical metrics and risks.

### FORMAT ###
- Exactly 4 bullet points
- Max 25 words each
- Start each with a number/percentage
- Include 1 risk point

### REPORT ###
{report}""",
    "✅ GOOD: Structured with constraints"
)

***Analysing Code Generation Quality With And Without Specifications***

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 2: CODE GENERATION QUALITY")
print("Focus: How prompt specificity affects code quality")
print("="*80)

compare(
    "Write a function to grade students based on their scores.",
    "❌ BAD: No logic or format specified"
)

compare(
    """Write a Python function:

NAME: get_student_grades

INPUT: A dictionary where keys are student names (str) and values are lists of test scores (List[int]).

REQUIREMENTS:
1. Calculate the average score for each student.
2. Assign a letter grade based on the average: 
   - 90+ : 'A'
   - 80-89: 'B'
   - 70-79: 'C'
   - Below 70: 'F'
3. Return a new dictionary: {student_name: grade_letter}.
4. If a student has no scores, assign them 'Incomplete'.
5. Include type hints, a docstring, and 2 test cases (one normal, one with an empty list).
6. The code must be ready to copy-paste and run immediately.
7. Output only the code block. Remove all markdown triple backticks.

OUTPUT: Complete runnable code only""",
    "✅ GOOD: Specific Logic and Handling Edge Cases"
)

***How Zero-Shot And Few-Shot Effect Output Format***

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 3: ZERO-SHOT VS FEW-SHOT LEARNING")
print("Focus: Impact of examples on classification accuracy")
print("="*80)

compare(
    """Classify sentiment: Positive, Negative, or Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:""",
    "ZERO-SHOT: No examples"
)

compare(
    """Classify sentiment considering BOTH product and service:

EXAMPLES:
"Amazing quality!": Positive
"Broke after one day": Negative  
"Product okay, slow shipping": Neutral
"Defective but great replacement service": Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:""",
    "FEW-SHOT: With examples"
)

***How Chain-Of-Thought Effects A Prompt In Giving Outputs***

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 4: CHAIN-OF-THOUGHT REASONING")
print("="*80)


math_problem = """A store has 156 laptops. Monday: sold 23% of inventory. 
Tuesday: received 85 new laptops. Wednesday: sold 42 laptops. 
How many laptops now?"""

compare(
    f"{math_problem}\n\nProvide only the final number.",
    "WITHOUT CoT : Direct answer requested"
)

compare(
    f"""{math_problem}

Let's solve step by step:
1. Calculate Monday's sales (23% of 156)
2. Remaining after Monday
3. Add Tuesday's shipment (+85)
4. Subtract Wednesday's sales (-42)
5. Final answer

Show calculations for each step.""",
    "WITH CoT"
)

***How Role Playing(Persona) Helps in Getting a Desired Output***

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 5: ROLE/PERSONA IMPACT")
print("="*80)

compare(
    "Explain machine learning in 3 sentences.",
    "NO ROLE"
)

compare(
    """You are a CS professor teaching freshmen with no coding background.

Explain machine learning in exactly 3 sentences:
1. What it is (use everyday analogy)
2. How it works (simple mechanism)  
3. Real-world example

Avoid jargon. Conversational tone.""",
    "WITH PROFESSOR ROLE"
)

compare(
    """You are explaining to a curious 8-year-old.

Explain machine learning using:
- Simple words only
- Comparison to toys/pets/games
- Max 3 short sentences
- Fun, enthusiastic tone""",
    "WITH CHILD-FRIENDLY ROLE"
)

***How can we Control The Output Format***

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 6: OUTPUT FORMAT CONTROL")
print("="*80)

text = """Sarah Johnson, 32, Senior Developer at TechCorp. 
Email: sarah.j@techcorp.com, Phone: +1-555-0123. 
Started 2019. Skills: Python, JavaScript. Reports to Mike Chen."""

compare(
    f"Extract information from:\n{text}",
    "NO FORMAT SPECIFIED"
)

compare(
    f"""Extract info and output ONLY valid JSON with this structure:

{{
  "name": "string",
  "age": number,
  "company": "string",
  "position": "string",
  "contact": {{"email": "string", "phone": "string"}},
  "employment": {{
    "start_year": number,
    "skills": ["string"],
    "manager": "string"
  }}
}}

TEXT: {text}

Return ONLY JSON, no markdown or extra text.""",
    "WITH JSON FORMAT"
)

***Resolving the Problem Of Ambiguity*** 

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT 7: AMBIGUITY RESOLUTION")
print("="*80)

compare(
    "Write about Python.",
    "AMBIGUOUS"
)

compare(
    """CONTEXT: I'm a Java developer learning Python programming.

TASK: Compare Java and Python syntax with 3 key differences.

FORMAT (for each):
- Feature name
- Java code example
- Python code example  
- 1-sentence explanation

CONSTRAINTS:
- Syntax only (not performance)
- Simple examples (variables, loops, functions)
- Max 5 lines of code each""",
    "DISAMBIGUATED"
)